# 2.1 Using Tools

In [1]:
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, ToolCall, ToolMessage
from typing import Literal
from langchain.tools import tool

In [2]:
llm = AzureChatOpenAI(model="gpt-4.1-mini")

In [3]:
@tool
def weather_tool(location: Literal["Chicago", "New York", "Los Angeles"]) -> str:
    """
    Get the current weather for a specified location.
    Args:
        location (str): The name of the city to get the weather for. 
                        Must be one of "Chicago", "New York", or "Los Angeles".
    Returns:
        str: A string describing the current weather in the specified location.
    """

    weather_data = {
        "New York": "Sunny, 25°C",
        "Los Angeles": "Cloudy, 22°C",
        "Chicago": "Rainy, 18°C"
    }
    return weather_data.get(location, "Weather data not available for this location.")

In [4]:
llm_with_tools = llm.bind_tools([weather_tool])

In [5]:
messages = [
    SystemMessage(
        content="You are a helpful assistant that can provide weather information for specific cities."
    ),
    HumanMessage(
        content="What is the weather like in NYC?"
    )
]

In [6]:
response = llm_with_tools.invoke(input=messages)
messages.append(response)

if len(response.tool_calls) > 0:
    for tool_call in response.tool_calls:
        messages.append(
            ToolMessage(
                name=tool_call["name"],
                content=weather_tool(tool_call["args"]["location"]),
                tool_call_id=tool_call["id"],
            )
        )
    response = llm_with_tools.invoke(input=messages)
    messages.append(response)

/var/folders/km/4c7v4jmj5ydfnj183qz7wr7w0000gn/T/ipykernel_99545/583047370.py:9: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  content=weather_tool(tool_call["args"]["location"]),


In [7]:
for msg in messages:
    msg.pretty_print()

================================ System Message ================================

You are a helpful assistant that can provide weather information for specific cities.
================================ Human Message =================================

What is the weather like in NYC?
================================== Ai Message ==================================
Tool Calls:
  weather_tool (call_rOvbNFN84jjonlWr8qiOFLcb)
 Call ID: call_rOvbNFN84jjonlWr8qiOFLcb
  Args:
    location: New York
================================= Tool Message =================================
Name: weather_tool

Sunny, 25°C
================================== Ai Message ==================================

The weather in New York City is currently sunny with a temperature of 25°C.
